1. Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?


Precision@k определяет какой процент из топ рекомендованных user купил, а hit_rate@k - был ли хотя бы 1 релевантный товар среди топ-k рекомендованных. Предпочительней, конечно precision@k, потому что он более информативен и говорит о бизнес-метрике, которую можно использовать в бизнесе.  hit_rate@k - может просто показать, что что-то купили из списка рекомендованных дорогих товаров (например, дорогих бриллиантов). Иногда нужно просто узнать было ли какое-то взаимодействие клиента с каким-то рекомендованным товаром или не было и в этом случае тоже удобно воспользоваться метрикой hit_rate.


2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.



Позиции в начале списка отличаются значительно сильнее чем в конце, логарифм в метрике NDCG@k позволяет учесть это обстоятельство. В ML в предобработке используется логарифм, когда данные очень сильно отличаются (в разы) и очень сложно их совместить для последующего анализа. 

3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

Метрики ранжирования AP@k, MAP@k, AUC@k, MRR@k, бизнес метрика - money precision

4. boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, MRR@8,  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

IDCG должен считаться для всех 8 позиций рекомендаций

In [49]:
import math
import numpy as np

In [50]:
boughted = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

1) pr@8 = (0+1+1+0+1+1+0+0)/8 = 0.5







In [55]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision


In [56]:
precision_at_k(recommended, boughted, k=8)

0.5

rec@8 = (0+1+1+0+1+1+0+0)/6 = 0.67

In [58]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [60]:
recall_at_k(recommended, boughted, k=8)

0.6666666666666666

AP@8 = (0+1/2+2/3+3/5+4/6)/8 = 0.3

In [75]:
def ap_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(recommended_list, bought_list)
       
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 
        if flags[i-1] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / len(flags)
    
    return result

In [76]:
ap_k(recommended, boughted, k=8)

0.30416666666666664

NDCG@8 = (0+1/log(2)+1/log(3)+0+1/log(5)+1/log(6)+0+0)/(1/1+1/log(2)+1/log(3)+1/log(4)+1/log(5)+1/log(6)+1/log(7)+1/log(8)= 0.21



In [44]:
def NDCG(recommended_list, bought_list):
    𝑖𝑑𝑒𝑎𝑙𝐷𝐶𝐺=0
    DCG=0
    Discount = 0
    for i, item_rec in enumerate(recommended_list):
        for item_bought in bought_list:
            if item_rec == item_bought:
                if i == 0:
                    Discount = 1
                else:
                    DCG += 1 / math.log2(i+1)
            DCG = DCG + Discount
        if i == 0:
                    idealDiscount = 1
        else:
            𝑖𝑑𝑒𝑎𝑙𝐷𝐶𝐺 += 1/math.log2(i+1)
        𝑖𝑑𝑒𝑎𝑙𝐷𝐶𝐺 = 𝑖𝑑𝑒𝑎𝑙𝐷𝐶𝐺 + idealDiscount
     
    return DCG / idealDCG

In [45]:
NDCG(recommended[:8], boughted)

0.2103848766764781

MRR@8 = (0+1/2+1/3+0+1/5+1/6+0+0)/8 = 0.15

In [25]:
def reciprocal_rank(recommended_list, bought_list):
    ranks=0.
    for i, item_rec in enumerate(recommended_list):
        for item_bought in bought_list:
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)

In [26]:
reciprocal_rank(recommended[:8], boughted)

0.15

**Пример выполнения 4го задания**

MSE на  
$y = [0, 2]$  
$y_{pred} = [1, 2]$

MSE = $0.5*((0 - 1)^2 + (2-2)^2) = 0.5$ 

In [4]:
import numpy as np
((np.array([0, 2]) - np.array([1, 2]))**2).mean()

0.5